In [4]:
import pandas as pd, numpy as np
import random
from pulp import *
from script.helper import read_form_response, prep_dirs
from ortoolpy import addbinvars

In [7]:
#12/30日当直は越山先生

# Parameters
year_plan, month_plan = 2026, 'ny'
lp_root = ['/home/atiroms/Documents','D:/atiro','D:/NICT_WS','/Users/smrt']
l_member_desig = ['藤川 慎也','熊倉 陽介','森田 進','水谷 真志','清田 正紘']
l_member_nondesig = ['田尻 智哉','佐藤 未悠','南學 正仁','相澤 里佳','押見 陽友']
l_member_oc = ['岡田 直大', '小池 進介', '榊原 英輔', '市橋 香代']
l_duty = ['12/26 当直','12/27 日当直','12/28 日当直','12/29 日当直','12/31 日当直','1/1 日当直','1/2 日当直','1/3 日当直','1/4 日当直']
l_rank = ['第1希望', '第2希望', '第3希望', '第4希望', '第5希望', '第6希望', '第7希望', '第8希望', '第9希望']

# Read G sheet
p_root, p_month, p_data = prep_dirs(lp_root, year_plan, month_plan, prefix_dir = '', make_data_dir = False)
path_form = '/dutyshift/result/2026/ny/form_2026ny'
d_preference_src = read_form_response(p_root, path_form)
# sort by timestamp
d_preference_src = d_preference_src.sort_values(by = 'Timestamp').reset_index(drop = True)
# Delete duplicated entries, keep the last one by timestamp
d_preference_src = d_preference_src.drop_duplicates(subset = ['お名前（敬称略）'], keep = 'last').reset_index(drop = True)

# Convert
def convert_preference(d_preference_src, l_member, l_duty, l_rank):    
    d_preference = pd.DataFrame(index = [i for i in range(len(l_duty))], columns = [i for i in range(len(l_member))])
    for i_member, name_member in enumerate(l_member):
        if name_member in d_preference_src['お名前（敬称略）'].tolist():
            s_preference_member = d_preference_src.loc[d_preference_src['お名前（敬称略）'] == name_member, [col.startswith('日当直希望') for col in d_preference_src.columns]]
            s_preference_member = s_preference_member.iloc[len(s_preference_member)-1, :]
            for i_duty, name_duty in enumerate(l_duty):
                rank_src = s_preference_member[[name_duty in i_pref for i_pref in s_preference_member.index.tolist()]].tolist()[0]
                for i_rank, name_rank in enumerate(l_rank):
                    if rank_src == name_rank:
                        d_preference.loc[i_duty, i_member] = i_rank
    return d_preference

# index: duty, col: member, value: rank of preference
d_preference_desig = convert_preference(d_preference_src, l_member_desig, l_duty, l_rank)
d_preference_nondesig = convert_preference(d_preference_src, l_member_nondesig, l_duty, l_rank)
d_preference_oc = convert_preference(d_preference_src, l_member_oc, l_duty, l_rank)

# Convert rank quadratically
def quadratic_conversion(d_preference):
    d_preference_conv = d_preference.copy()
    for i in reversed(range(len(d_preference))):
        rank_pre = i
        rank_post = i * (i + 1)/2
        d_preference_conv = d_preference_conv.replace(rank_pre, rank_post)
    return d_preference_conv

d_preference_desig = quadratic_conversion(d_preference_desig)
d_preference_nondesig = quadratic_conversion(d_preference_nondesig)
d_preference_oc = quadratic_conversion(d_preference_oc)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=286844707417-54por1avpn0r7ulus5qoa1fc3mb0jjhm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55863%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&state=MioQEBygZnoZkwJCQrIeNSW2HP5OYP&access_type=offline


In [11]:
# Optimize assignment

# Initialize model to be optimized
prob_assign = LpProblem()

# Binary assignment variables to be optimized, index: duty, column: member
l_member = l_member_desig + l_member_nondesig + l_member_oc
dv_assign = pd.DataFrame(np.array(addbinvars(len(l_duty), len(l_member))))

# Variable dataframe for calculating designation and oc
# designated doctor = 3, non-designated doctor = 2, oc = 1. sum of each duty = 3
dv_member_type = pd.DataFrame(index = [i for i in range(len(l_duty))], columns = [i for i in range(len(l_member))])
for i_member, name_member in enumerate(l_member):
    if name_member in l_member_desig:
        dv_member_type.iloc[:, i_member] = 3 * dv_assign.iloc[:, i_member]
    elif name_member in l_member_nondesig:
        dv_member_type.iloc[:, i_member] = 2 * dv_assign.iloc[:, i_member]
    else:
        dv_member_type.iloc[:, i_member] = 1 * dv_assign.iloc[:, i_member]
for i_duty in range(len(l_duty)):
    prob_assign += lpSum(dv_member_type.iloc[i_duty, :]) == 3

# Each member is assigned to at most one duty
for i_member in range(len(l_member)):
    prob_assign += lpSum(dv_assign.iloc[:, i_member]) <= 1

# Objective function: minimize total preference score
obj_assign = lpSum([
    dv_assign.iloc[i_duty, i_member] * (d_preference_desig.iloc[i_duty, i_member] if i_member < len(l_member_desig) else
                           d_preference_nondesig.iloc[i_duty, i_member - len(l_member_desig)] if i_member < len(l_member_desig) + len(l_member_nondesig) else
                           d_preference_oc.iloc[i_duty, i_member - len(l_member_desig) - len(l_member_nondesig)])
    for i_duty in range(len(l_duty)) for i_member in range(len(l_member))
])
prob_assign += obj_assign

# Solve
prob_assign.solve()

# Output assignment result
duty_assignment = pd.DataFrame(index = l_duty, columns = ['日当直','オンコール'])
for i_duty in range(len(l_duty)):
    for i_member in range(len(l_member)):
        if dv_assign.iloc[i_duty, i_member].varValue == 1:
            member = l_member[i_member]
            if member in (l_member_desig + l_member_nondesig):
                duty_assignment.iloc[i_duty, 0] = member
            else:
                duty_assignment.iloc[i_duty, 1] = member


print(duty_assignment)

             日当直  オンコール
12/26 当直   押見 陽友  榊原 英輔
12/27 日当直  南學 正仁  市橋 香代
12/28 日当直  田尻 智哉  小池 進介
12/29 日当直  藤川 慎也    NaN
12/31 日当直  水谷 真志    NaN
1/1 日当直    相澤 里佳  岡田 直大
1/2 日当直    清田 正紘    NaN
1/3 日当直    熊倉 陽介    NaN
1/4 日当直     森田 進    NaN
